# Topic Modeling on Each Cluster

## Libraries Needed

In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import string
import os

In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


## Load Data

In [3]:
df_covid = pickle.load(open("plot_data/df_covid_new.p", "rb"))

y_pred = pickle.load(open("plot_data/y_pred_new.p", "rb"))

# kmeans = pickle.load(open("plot_data/kmeans.p", "rb"))

In [4]:
df_covid = df_covid.loc[df_covid['language'] == 'en']

In [5]:
df_covid['y'] = y_pred

In [6]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24934 entries, 0 to 27677
Data columns (total 14 columns):
paper_id               24934 non-null object
abstract               24934 non-null object
body_text              24934 non-null object
authors                24934 non-null object
title                  24934 non-null object
journal                24934 non-null object
abstract_summary       24934 non-null object
abstract_word_count    24934 non-null int64
body_word_count        24934 non-null int64
body_text_clean        24934 non-null object
abstract_clean         24934 non-null object
processed_text         24934 non-null object
language               24934 non-null object
y                      24934 non-null int32
dtypes: int32(1), int64(2), object(11)
memory usage: 2.8+ MB


## Example

In [7]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [8]:
df_covid["abstract"][23]

'Bordetella bronchiseptica isolate KM22 has been used in experimental infections of swine as a model of clinical B. bronchiseptica infection and to study host-to-host transmission. The draft genome sequence of KM22 was reported in 2014. Here, we report the complete genome sequence of KM22.'

In [9]:
doc = nlp(df_covid["abstract"][20])

In [10]:
# Lemmatization
review = str(" ".join([i.lemma_ for i in doc]))

In [11]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

## Vectorize for Each Cluster using CountVectorizer

In [12]:
vectorizers = []
    
for ii in range(0, 20):
    # Creating a vectorizer
    vectorizers.append(CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}'))

In [13]:
vectorizers[0]

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.9, max_features=None, min_df=5,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='[a-zA-Z\\-][a-zA-Z\\-]{2,}',
                tokenizer=None, vocabulary=None)

In [14]:
vectorized_data = []

current_cluster = 0
for cvec in vectorizers:
    try:
        vectorized_data.append(cvec.fit_transform(df_covid.loc[df_covid['y'] == current_cluster, 'processed_text']))
    except Exception as e:
        print("Not enough instances in cluster: " + str(current_cluster))
        pass
    current_cluster += 1

In [15]:
vectorized_data[0]

<1089x12244 sparse matrix of type '<class 'numpy.int64'>'
	with 751035 stored elements in Compressed Sparse Row format>

In [16]:
len(vectorized_data)

20

## Get the Topics Per Cluster Using LatentDirichletAllocation

In [17]:
# number of topics per cluster
NUM_TOPICS_PER_CLUSTER = 20

In [18]:
lda_models = []

for ii in range(0, 20):
    # Latent Dirichlet Allocation Model
    lda = LatentDirichletAllocation(n_components=NUM_TOPICS_PER_CLUSTER, max_iter=10, learning_method='online',verbose=False)
    lda_models.append(lda)
    
lda_models[0]

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=False)

In [19]:
clusters_lda_data = []

current_cluster = 0
for lda in lda_models:
    clusters_lda_data.append((lda.fit_transform(vectorized_data[current_cluster])))
    current_cluster+=1

## Get the Keywords from Each Topic on Each Cluster

In [20]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=2):
    current_words = []
    keywords = []
    
    for idx, topic in enumerate(model.components_):
        words = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]]
        for word in words:
            if word[0] not in current_words:
                keywords.append(word)
                current_words.append(word[0])
                
    keywords.sort(key = lambda x: x[1])  
    keywords.reverse()
    return_values = []
    for ii in keywords:
        return_values.append(ii[0])
    return return_values

In [21]:
current_vectorizer = 0
all_keywords = []
for lda in lda_models:
    #print("Current Cluster: " + str(current_vectorizer) +"\n")
    
    #if current_vectorizer == 14:
    #    current_vectorizer += 1
        #print("Current Cluster Changed to (14 is empty): " + str(current_vectorizer))
    all_keywords.append(selected_topics(lda, vectorizers[current_vectorizer]))
    current_vectorizer += 1
    #print("____________________________________________________________\n")

In [22]:
all_keywords[0][:10]

['ifn',
 'mouse',
 'protein',
 'type',
 'hbv',
 'fig',
 'virus',
 'patient',
 'cns',
 'rig']

In [23]:
len(all_keywords)

20

## Output to a Text File

Note that cluster 14 didn't have enough instances to get the keywords.

In [25]:
f=open('lib/topics.txt','w')

count = 0
for ii in all_keywords:
    #if count == 14:
    #    f.write("Not enough instances to be determined. \n")
    #    f.write(', '.join(ii) + "\n")
    #else:
    f.write(', '.join(ii) + "\n")
    count += 1
f.close()    